In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [3]:
DATA_DIR = os.path.join(PROJECT_ROOT_DIR,"datasets","jsb_chorales")

train_files = [os.path.join(DATA_DIR,"train",x) for x in os.listdir(os.path.join(DATA_DIR,"train"))]
test_files = [os.path.join(DATA_DIR,"test",x) for x in os.listdir(os.path.join(DATA_DIR,"test"))]
valid_files = [os.path.join(DATA_DIR,"valid",x) for x in os.listdir(os.path.join(DATA_DIR,"valid"))]
train_files

['.\\datasets\\jsb_chorales\\train\\chorale_000.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_001.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_002.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_003.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_004.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_005.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_006.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_007.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_008.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_009.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_010.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_011.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_012.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_013.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_014.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_015.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_016.csv',
 '.\\datasets\\jsb_chorales\\train\\chorale_017.csv',
 '.\\datasets\\jsb_chorales\

In [4]:
def preprocess(line):
    defs = [0] * 4 + [tf.constant([], dtype=tf.float32)]
    print("Defaults ",defs)
    fields = tf.io.decode_csv(line, record_defaults=defs)
    

In [5]:
train_files_set = tf.data.Dataset.list_files(train_files, seed=13)
valid_files_set = tf.data.Dataset.list_files(valid_files, seed=13) 

In [12]:
import pandas as pd

sample = pd.read_csv(train_files[1])
train_files_set

<DatasetV1Adapter shapes: (), types: tf.string>

In [7]:
print("training set size: ",len(train_files))
print("validation set size: ",len(valid_files))
print("test set size: ",len(test_files))

training set size:  229
validation set size:  76
test set size:  77


In [40]:
# TODO: need to chop each file into batches of time steps?
# use tf.data.Dataset.window to create windows.
# Split chords into sequences of tones to improve harmonic relations
# Use embeddings to get better harmonic relations

# Lowest note is 36, highest is 81
# Shift notes by the smallest note to get values from 1(0)-46
# Keep silences (=0) as they are
def preprocess_chords(chords):
    chords = tf.reshape(chords, [-1])
    chords.where(chords==0, chords, chords - 35)
    
def extract_target(batch):
    X = batch[:, :-1]
    Y = batch[:, 1:]
    return X, Y
    
def preprocess_and_batch(chorales, window_size=32, window_shift=16, batch_size=32):
    
    # Helper function for creating windows, batching and returning in flattened form
    def create_window(chorales):
        dataset = tf.data.Dataset.from_tensor_slices(chorales)
        dataset = dataset.window(window_size+1,window_shift,1,True)
        return dataset.flat_map(lambda x: Dataset.batch(x))
    
    # Create ragged tensor from data, reshape into 1D array and apply preprocessing,
    # finally batch the data and extract target steps
    chorales = tf.ragged.constant(chorales, ragged_rank=1)
    print("chorales:",chorales)
    chorale_ds = tf.data.Dataset.from_tensor_slices(chorales)
    print(chorale_ds.element_spec)
    chorale_ds = chorale_ds.flat_map(create_window).map(preprocess_chords)                              
    chorale_ds = chorale_ds.batch(batch_size)
    chorale_ds = chorale_ds.map(extract_target)
    
    return chorale_ds.prefetch(1)

In [22]:
def open_data_files(file_paths):
    return [pd.read_csv(file_path).values.tolist() for file_path in file_paths]

# TODO: load all sets with function above
train_data = open_data_files(train_files)
valid_data = open_data_files(valid_files)
test_data = open_data_files(test_files)

train_data[0]

[[74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [74, 70, 65, 58],
 [75, 70, 58, 55],
 [75, 70, 58, 55],
 [75, 70, 60, 55],
 [75, 70, 60, 55],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 69, 62, 50],
 [77, 70, 62, 55],
 [77, 70, 62, 55],
 [77, 69, 62, 55],
 [77, 69, 62, 55],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [72, 69, 65, 53],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [74, 70, 65, 46],
 [75, 69, 63, 48],
 [75, 69, 63, 48],
 [75, 67, 63, 48],
 [75, 67, 63, 48],
 [77, 65, 62, 50],
 [77, 65, 62, 50],
 [77, 65, 60, 50],
 [77, 65, 60, 50],
 [74, 67, 58, 55],
 [74, 67, 58, 55],
 [74, 67, 58, 53],
 [74, 67, 58, 53],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 67, 58, 51],
 [72, 65, 57

## Baseline generation
### Collect data about distribution of values

In [8]:
data_sample = pd.read_csv(train_files[0])
cond = data_sample > 0
data_sample.where(cond,inplace=True)
lengths = []
lengths.append(data_sample.shape[0])

for file in train_files[1:]:
    f = pd.read_csv(file)
    cond = f > 0
    f.where(cond,inplace=True)
    lengths.append(f.shape[0])
    data_sample = data_sample.append(f)
    print(data_sample.shape)

(420, 4)
(628, 4)
(1060, 4)
(1320, 4)
(1532, 4)
(1824, 4)
(2004, 4)
(2136, 4)
(2328, 4)
(2536, 4)
(2808, 4)
(3068, 4)
(3200, 4)
(3396, 4)
(3852, 4)
(4060, 4)
(4512, 4)
(4864, 4)
(5060, 4)
(5256, 4)
(5480, 4)
(5676, 4)
(6060, 4)
(6256, 4)
(6648, 4)
(6860, 4)
(7072, 4)
(7236, 4)
(7432, 4)
(7608, 4)
(7740, 4)
(7920, 4)
(8020, 4)
(8252, 4)
(8512, 4)
(8736, 4)
(8896, 4)
(9204, 4)
(9464, 4)
(9692, 4)
(10032, 4)
(10224, 4)
(10556, 4)
(10928, 4)
(11232, 4)
(11568, 4)
(11808, 4)
(12016, 4)
(12164, 4)
(12568, 4)
(12812, 4)
(13040, 4)
(13172, 4)
(13304, 4)
(13576, 4)
(13836, 4)
(14096, 4)
(14372, 4)
(14504, 4)
(14764, 4)
(14960, 4)
(15156, 4)
(15384, 4)
(15564, 4)
(15792, 4)
(15984, 4)
(16140, 4)
(16336, 4)
(16564, 4)
(16996, 4)
(17128, 4)
(17464, 4)
(17768, 4)
(17932, 4)
(18448, 4)
(18676, 4)
(18868, 4)
(19000, 4)
(19240, 4)
(19448, 4)
(19676, 4)
(19920, 4)
(20180, 4)
(20440, 4)
(20744, 4)
(20876, 4)
(21296, 4)
(21524, 4)
(21688, 4)
(22016, 4)
(22276, 4)
(22472, 4)
(22700, 4)
(22832, 4)
(23052, 

In [9]:
pd.Series(lengths).describe()

count    229.000000
mean     241.170306
std       78.130524
min      100.000000
25%      192.000000
50%      228.000000
75%      260.000000
max      516.000000
dtype: float64

In [10]:
data_sample.describe()

,note0,note1,note2,note3
count,55155.000000,55132.000000,55110.000000,55104.000000
mean,70.416263,64.979903,59.462856,50.637921
std,3.672974,3.192862,3.328952,4.764402
min,60.000000,52.000000,46.000000,36.000000
25%,68.000000,62.000000,57.000000,47.000000
50%,71.000000,65.000000,59.000000,50.000000
75%,73.000000,67.000000,62.000000,54.000000
max,81.000000,74.000000,69.000000,66.000000


### Random generator which uses collected data as parameters

In [9]:
rng = np.random.default_rng()

def generate_random_bach(n_steps):
    #length = rng.integers(100, 517)
    n0 = rng.integers(60, 82, size=n_steps)
    n1 = rng.integers(52, 75, size=n_steps)    
    n2 = rng.integers(46, 70, size=n_steps)    
    n3 = rng.integers(36, 67, size=n_steps)    
    
    generated = np.stack((n0,n1,n2,n3),axis=-1)

    return generated

In [10]:
gen1 = generate_random_bach(10)
gen1.shape

(10, 4)

## First simple RNN model

In [9]:
import keras

simple_seq = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None,4]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.Dense(1)
])

Using TensorFlow backend.


In [41]:
train_prep = preprocess_and_batch(train_data)
valid_prep = preprocess_and_batch(valid_data)
test_prep = preprocess_and_batch(test_data)
train_prep

chorales: <tf.RaggedTensor [[[74, 70, 65, 58], [74, 70, 65, 58], [74, 70, 65, 58], [74, 70, 65, 58], [75, 70, 58, 55], [75, 70, 58, 55], [75, 70, 60, 55], [75, 70, 60, 55], [77, 69, 62, 50], [77, 69, 62, 50], [77, 69, 62, 50], [77, 69, 62, 50], [77, 70, 62, 55], [77, 70, 62, 55], [77, 69, 62, 55], [77, 69, 62, 55], [75, 67, 63, 48], [75, 67, 63, 48], [75, 69, 63, 48], [75, 69, 63, 48], [74, 70, 65, 46], [74, 70, 65, 46], [74, 70, 65, 46], [74, 70, 65, 46], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [72, 69, 65, 53], [74, 70, 65, 46], [74, 70, 65, 46], [74, 70, 65, 46], [74, 70, 65, 46], [75, 69, 63, 48], [75, 69, 63, 48], [75, 67, 63, 48], [75, 67, 63, 48], [77, 65, 62, 50], [77, 65, 62, 50], [77, 65, 60, 50], [77, 65, 60, 50], [74, 67, 58, 55], [74, 67, 58, 55], [74, 67, 58, 53], [74, 67, 58, 53], [72, 67, 58, 51], [72, 67, 58, 51], [72, 67, 58, 51], [72, 67, 58, 51], [72, 65, 57, 53], [72, 65, 57, 53],

ValueError: ragged_rank must be non-negative; got 0.